# 3 Data Driven Decision Making with advanced SQL queries

The concept of nested queries and correlated nested queries is introduced and the functions EXISTS and UNION are used to categorize customers, movies, actors, and more.

# Often rented movies

Your manager wants you to make a list of movies excluding those which are hardly ever watched. This list of movies will be used for advertising. List all movies with more than 5 views using a nested query which is a powerful tool to implement selection conditions.

# Instructions:

- Select all movie IDs which have more than 5 views.

In [ ]:
SELECT movie_id -- Select movie IDs with more than 5 views
FROM renting
GROUP BY movie_id
HAVING COUNT(*) > 5;

- Select all information about movies with more than 5 views.

In [ ]:
SELECT *
FROM movies
WHERE movie_id IN -- Select movie IDs from the inner query
	(SELECT movie_id
	FROM renting
	GROUP BY movie_id
	HAVING COUNT(*) > 5)

# Frequent customers

Report a list of customers who frequently rent movies on MovieNow.

# Instructions:

- List all customer information for customers who rented more than 10 movies.

In [ ]:
SELECT *
FROM customers
WHERE customer_id IN -- Select all customers with more than 10 movie rentals
	(SELECT customer_id
	FROM renting
	GROUP BY customer_id
	HAVING COUNT(*) > 10);

# Movies with rating above average

For the advertising campaign your manager also needs a list of popular movies with high ratings. Report a list of movies with rating above average.

# Instructions:

- Calculate the average over all ratings.

In [ ]:
SELECT AVG(rating) -- Calculate the total average rating
FROM renting;

- Select movie IDs and calculate the average rating of movies with rating above average.

In [ ]:
SELECT movie_id,  -- Select movie IDs and calculate the average rating 
       AVG(rating)
FROM renting
GROUP BY movie_id
HAVING AVG(rating) >   -- Of movies with rating above average
	(SELECT AVG(rating)
	FROM renting);

- The advertising team only wants a list of movie titles. Report the movie titles of all movies with average rating higher than the total average.

In [ ]:
SELECT title -- Report the movie titles of all movies with average rating higher than the total average
FROM movies
WHERE movie_id IN
	(SELECT movie_id
	 FROM renting
     GROUP BY movie_id
     HAVING AVG(rating) > 
		(SELECT AVG(rating)
		 FROM renting));

# Analyzing customer behavior

A new advertising campaign is going to focus on customers who rented fewer than 5 movies. Use a correlated query to extract all customer information for the customers of interest.

# Instructions:

- First, count number of movie rentals for customer with customer_id=45. Give the table renting the alias r.

In [ ]:
-- Count movie rentals of customer 45
SELECT COUNT(*)
FROM renting as r
WHERE r.customer_id = 45;

- Now select all columns from the customer table where the number of movie rentals is smaller than 5.

In [ ]:
-- Select customers with less than 5 movie rentals
SELECT *
FROM customers as c
WHERE 5 > 
	(SELECT count(*)
	FROM renting as r
	WHERE r.customer_id=c.customer_id);

# Customers who gave low ratings

Identify customers who were not satisfied with movies they watched on MovieNow. Report a list of customers with minimum rating smaller than 4.

# Instructions:

- Calculate the minimum rating of customer with ID 7.

In [ ]:
-- Calculate the minimum rating of customer with ID 7
SELECT MIN(rating)
FROM renting
WHERE customer_id = 7;

- Select all customers with a minimum rating smaller than 4. Use the first letter of the table as an alias.

In [ ]:
SELECT *
FROM customers AS c
WHERE 4 >  -- Select all customers with a minimum rating smaller than 4
	(SELECT MIN(rating)
	FROM renting AS r
	WHERE r.customer_id = c.customer_id);

# Movies and ratings with correlated queries

Report a list of movies that received the most attention on the movie platform, (i.e. report all movies with more than 5 ratings and all movies with an average rating higher than 8).

# Instructions:

- Select all movies with more than 5 ratings. Use the first letter of the table as an alias.

In [ ]:
SELECT *
FROM movies AS m
WHERE 5 < -- Select all movies with more than 5 ratings
	(SELECT COUNT(rating)
	FROM renting AS r
	WHERE r.movie_id = m.movie_id);

- Select all movies with an average rating higher than 8.

In [ ]:
SELECT *
FROM movies AS m
WHERE 8 <  -- Select all movies with an average rating higher than 8
	(SELECT AVG(rating)
	FROM renting AS r
	WHERE r.movie_id = m.movie_id);

# Customers with at least one rating

Having active customers is a key performance indicator for MovieNow. Make a list of customers who gave at least one rating.

# Instructions:

- Select all records of movie rentals from customer with ID 115.



In [ ]:
-- Select all records of movie rentals from customer with ID 115
SELECT *
FROM renting
WHERE customer_id = 115;

- Select all records of movie rentals from the customer with ID 115 and exclude records with null ratings.

In [ ]:
SELECT *
FROM renting
WHERE rating IS NOT NULL -- Exclude those with null ratings
AND customer_id = 115;

- Select all records of movie rentals from the customer with ID 1, excluding null ratings.

In [ ]:
SELECT *
FROM renting
WHERE rating IS NOT NULL -- Exclude null ratings
AND customer_id = 1; -- Select all ratings from customer with ID 1

- Select all customers with at least one rating. Use the first letter of the table as an alias.

In [ ]:
SELECT *
FROM customers AS c -- Select all customers with at least one rating
WHERE EXISTS
	(SELECT *
	FROM renting AS r
	WHERE rating IS NOT NULL 
	AND r.customer_id = c.customer_id);

# Actors in comedies

In order to analyze the diversity of actors in comedies, first, report a list of actors who play in comedies and then, the number of actors for each nationality playing in comedies.

# Instructions:

- Select the records from the table actsin of all actors who play in a Comedy. Use the first letter of the table as an alias.

In [ ]:
SELECT *  -- Select the records from the table `actsin` of all actors who play in a Comedy
FROM actsin AS ai
LEFT JOIN movies AS m
ON ai.movie_id = m.movie_id
WHERE m.genre = 'Comedy';

- Make a table of the records of actors who play in a Comedy and select only the actor with ID 1.

In [ ]:
SELECT *
FROM actsin AS ai
LEFT JOIN movies AS m
ON m.movie_id = ai.movie_id
WHERE m.genre = 'Comedy'
AND ai.actor_id = 1; -- Select only the actor with ID 1

- Create a list of all actors who play in a Comedy. Use the first letter of the table as an alias.

In [ ]:
SELECT *
FROM actors AS a
WHERE EXISTS
	(SELECT *
	 FROM actsin AS ai
	 LEFT JOIN movies AS m
	 ON m.movie_id = ai.movie_id
	 WHERE m.genre = 'Comedy'
	 AND ai.actor_id = a.actor_id);

- Report the nationality and the number of actors for each nationality.

In [ ]:
SELECT a.nationality,  -- Report the nationality and number of actors for each nationality
COUNT(*) 
FROM actors AS a
WHERE EXISTS
	(SELECT ai.actor_id
	 FROM actsin AS ai
	 LEFT JOIN movies AS m
	 ON m.movie_id = ai.movie_id
	 WHERE m.genre = 'Comedy'
	 AND ai.actor_id = a.actor_id)
GROUP BY a.nationality;

# Young actors not coming from the USA

As you've just seen, the operators UNION and INTERSECT are powerful tools when you work with two or more tables. Identify actors who are not from the USA and actors who were born after 1990.

# Instructions:

- Report the name, nationality and the year of birth of all actors who are not from the USA.

In [ ]:
SELECT name,   -- Report the name, nationality and the year of birth
       nationality, 
       year_of_birth
FROM actors
WHERE nationality <> 'USA'; -- Of all actors who are not from the USA

- Report the name, nationality and the year of birth of all actors who were born after 1990.

In [ ]:
SELECT name, 
       nationality, 
       year_of_birth
FROM actors
WHERE year_of_birth > 1990; -- Born after 1990

- Select all actors who are not from the USA and all actors who are born after 1990.

In [ ]:
SELECT name, 
       nationality, 
       year_of_birth
FROM actors
WHERE nationality <> 'USA'
UNION -- Select all actors who are not from the USA and all actors who are born after 1990
SELECT name, 
       nationality, 
       year_of_birth
FROM actors
WHERE year_of_birth > 1990;

- Select all actors who are not from the USA and who are also born after 1990.

In [ ]:
SELECT name, 
       nationality, 
       year_of_birth
FROM actors
WHERE nationality <> 'USA'
INTERSECT -- Select all actors who are not from the USA and who are also born after 1990
SELECT name, 
       nationality, 
       year_of_birth
FROM actors
WHERE year_of_birth > 1990;

# Dramas with high ratings

The advertising team has a new focus. They want to draw the attention of the customers to dramas. Make a list of all movies that are in the drama genre and have an average rating higher than 9.

# Instructions:

- Select the IDs of all dramas.

In [ ]:
SELECT movie_id -- Select the IDs of all dramas
FROM movies
WHERE genre = 'Drama';

- Select the IDs of all movies with average rating higher than 9.

In [ ]:
SELECT movie_id -- Select the IDs of all movies with average rating higher than 9
FROM renting
GROUP BY movie_id
HAVING AVG(rating) > 9;

- Select the IDs of all dramas with average rating higher than 9.

In [ ]:
SELECT movie_id
FROM movies
WHERE genre = 'Drama'
INTERSECT  -- Select the IDs of all dramas with average rating higher than 9
SELECT movie_id
FROM renting
GROUP BY movie_id
HAVING AVG(rating)>9;

- Select all movies of in the drama genre with an average rating higher than 9.

In [ ]:
SELECT *
FROM movies
WHERE movie_id IN -- Select all movies of genre drama with average rating higher than 9
   (SELECT movie_id
    FROM movies
    WHERE genre = 'Drama'
    INTERSECT
    SELECT movie_id
    FROM renting
    GROUP BY movie_id
    HAVING AVG(rating)>9);